# Sleep Scoring Model Single File
Training a model on manual sleep scoring for a single file.

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import run_yasa
import logging
import mne
import yasa
import os
import argparse
import os

import mne
import numpy as np
import pandas as pd
from brainflow.board_shim import BoardShim, BoardIds
from brainflow.data_filter import DataFilter
log = lambda msg: logging.info(msg)


# Load data

In [2]:
input_file = "C:\\dev\\play\\brainwave-data\\2024-07-14-22-27-26\\raw.fif"
input_file_without_ext = os.path.splitext(input_file)[0]

In [3]:
import run_feature_pipeline
yasa_df = run_feature_pipeline.cached_pipeline(log, input_file)
yasa_df

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,...,Fpz-M1_eeg_std,Fpz-M1_eeg_std_c7min_norm,Fpz-M1_eeg_std_p2min_norm,Fpz-M1_eeg_theta,Fpz-M1_eeg_theta_c7min_norm,Fpz-M1_eeg_theta_p2min_norm,YASAProbablyAwake,DefinitelyAwake,ProbablyAwake,ProbablySleep
0,W,0.998098,0,2024-09-01 21:11:15.609066010+01:00,Fpz-M1,W,0.998098,0,1.924170e-09,0.674103,...,0.000061,0.408875,0.114821,0.143303,-0.031615,0.281790,False,False,False,False
1,W,0.998786,1,2024-09-01 21:11:45.609066010+01:00,Fpz-M1,W,0.998786,0,1.538265e-09,0.742847,...,0.000072,0.457701,0.133877,0.091520,-0.061840,0.096633,True,False,True,False
2,W,0.998694,2,2024-09-01 21:12:15.609066010+01:00,Fpz-M1,W,0.998694,0,3.222337e-09,0.804278,...,0.000268,0.498781,0.366230,0.098994,-0.076986,0.052729,True,False,True,False
3,W,0.997929,3,2024-09-01 21:12:45.609066010+01:00,Fpz-M1,W,0.997929,0,1.632114e-09,0.837591,...,0.000069,0.495865,0.310115,0.090746,-0.092663,0.016031,True,False,True,False
4,W,0.999127,4,2024-09-01 21:13:15.609066010+01:00,Fpz-M1,W,0.999127,0,1.673459e-09,0.880257,...,0.000053,0.505479,0.303755,0.068711,-0.108333,-0.117325,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,W,0.974482,1089,2024-09-02 06:15:45.609066010+01:00,Fpz-M1,W,0.974482,0,5.987828e-11,0.149338,...,0.000009,0.731613,0.456701,0.136710,0.079478,-0.051760,True,False,True,False
1090,W,0.989291,1090,2024-09-02 06:16:15.609066010+01:00,Fpz-M1,W,0.989291,0,6.303612e-11,0.172236,...,0.000009,0.854302,0.230547,0.291684,0.105994,0.424371,True,False,True,False
1091,W,0.911434,1091,2024-09-02 06:16:45.609066010+01:00,Fpz-M1,W,0.911434,0,1.675323e-10,0.204585,...,0.000454,1.021654,0.320756,0.128080,0.077381,0.643232,True,False,True,False
1092,W,0.988893,1092,2024-09-02 06:17:15.609066010+01:00,Fpz-M1,W,0.988893,0,3.977211e-09,0.252664,...,0.000230,1.151942,0.510719,0.006493,0.021929,0.263471,True,False,True,False


In [4]:
import json
import pandas as pd

json_file_path = input_file.replace('.fif', '.scorings.json')
with open(json_file_path, 'r') as file:
    data = json.load(file)
df = pd.json_normalize(data, 'scorings', errors='ignore')
df['tags'] = df['tags'].apply(lambda x: [tag['tag'] for tag in x] if isinstance(x, list) else [])
df.set_index('epochIndex', inplace=True)
df.rename(columns={'stage': 'ManualStage'}, inplace=True)
df

,scoredAt,ManualStage,tags
epochIndex,,,
0,2024-09-02T17:12:24.695Z,Wake,[Blinks]
1,2024-09-02T17:12:24.791Z,Wake,[Blinks]
2,2024-09-02T17:12:26.099Z,Wake,[Blinks]
3,2024-09-02T17:12:57.334Z,Wake,[Blinks]
4,2024-09-02T17:13:03.132Z,Wake,[Blinks]
...,...,...,...
1089,2024-09-02T18:35:34.392Z,Unsure,[]
1090,2024-09-02T18:35:35.059Z,Unsure,[]
1091,2024-09-02T18:35:40.593Z,Unsure,[MicrowakingComplete]


In [5]:
joined = yasa_df.join(df)
joined

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,...,Fpz-M1_eeg_theta,Fpz-M1_eeg_theta_c7min_norm,Fpz-M1_eeg_theta_p2min_norm,YASAProbablyAwake,DefinitelyAwake,ProbablyAwake,ProbablySleep,scoredAt,ManualStage,tags
0,W,0.998098,0,2024-09-01 21:11:15.609066010+01:00,Fpz-M1,W,0.998098,0,1.924170e-09,0.674103,...,0.143303,-0.031615,0.281790,False,False,False,False,2024-09-02T17:12:24.695Z,Wake,[Blinks]
1,W,0.998786,1,2024-09-01 21:11:45.609066010+01:00,Fpz-M1,W,0.998786,0,1.538265e-09,0.742847,...,0.091520,-0.061840,0.096633,True,False,True,False,2024-09-02T17:12:24.791Z,Wake,[Blinks]
2,W,0.998694,2,2024-09-01 21:12:15.609066010+01:00,Fpz-M1,W,0.998694,0,3.222337e-09,0.804278,...,0.098994,-0.076986,0.052729,True,False,True,False,2024-09-02T17:12:26.099Z,Wake,[Blinks]
3,W,0.997929,3,2024-09-01 21:12:45.609066010+01:00,Fpz-M1,W,0.997929,0,1.632114e-09,0.837591,...,0.090746,-0.092663,0.016031,True,False,True,False,2024-09-02T17:12:57.334Z,Wake,[Blinks]
4,W,0.999127,4,2024-09-01 21:13:15.609066010+01:00,Fpz-M1,W,0.999127,0,1.673459e-09,0.880257,...,0.068711,-0.108333,-0.117325,True,False,True,False,2024-09-02T17:13:03.132Z,Wake,[Blinks]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,W,0.974482,1089,2024-09-02 06:15:45.609066010+01:00,Fpz-M1,W,0.974482,0,5.987828e-11,0.149338,...,0.136710,0.079478,-0.051760,True,False,True,False,2024-09-02T18:35:34.392Z,Unsure,[]
1090,W,0.989291,1090,2024-09-02 06:16:15.609066010+01:00,Fpz-M1,W,0.989291,0,6.303612e-11,0.172236,...,0.291684,0.105994,0.424371,True,False,True,False,2024-09-02T18:35:35.059Z,Unsure,[]
1091,W,0.911434,1091,2024-09-02 06:16:45.609066010+01:00,Fpz-M1,W,0.911434,0,1.675323e-10,0.204585,...,0.128080,0.077381,0.643232,True,False,True,False,2024-09-02T18:35:40.593Z,Unsure,[MicrowakingComplete]
1092,W,0.988893,1092,2024-09-02 06:17:15.609066010+01:00,Fpz-M1,W,0.988893,0,3.977211e-09,0.252664,...,0.006493,0.021929,0.263471,True,False,True,False,2024-09-02T18:35:42.863Z,Wake,[]


In [6]:
joined['tags'].value_counts()

tags
[]                                                 872
[Possible non-wake disturbance]                    104
[Microwaking]                                       29
[MicrowakingComplete]                               24
[MicrowakingEnd]                                    16
[Weak deep]                                         14
[Blinks]                                            12
[Microwaking, MicrowakingEnd]                        7
[Strong deep]                                        5
[MicrowakingEnd, Possible non-wake disturbance]      3
[Unusual]                                            1
[Possible non-wake disturbance, Weak deep]           1
[Microwaking, Possible non-wake disturbance]         1
[Uniform]                                            1
Name: count, dtype: int64

# Preparing data for model

In [7]:
joined['ManualStage'].value_counts()

ManualStage
Non-Deep          658
Deep              165
Wake              157
Ambiguous Deep     56
Unsure             54
Name: count, dtype: int64

In [8]:
import manual_sleep_scoring_catboost_1
prepared_df, X, y = manual_sleep_scoring_catboost_1.pipeline(joined)

display(prepared_df)
display(X)
display(y)

C:\dev\play\brainwave-processor\models\manual_sleep_scoring_catboost_1\manual_sleep_scoring_catboost_1.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[boolean_columns] = X[boolean_columns].astype(int)


,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_at,Fpz-M1_eeg_at_c7min_norm,Fpz-M1_eeg_at_p2min_norm,Fpz-M1_eeg_auc,...,Fpz-M1_eeg_theta_c7min_norm,Fpz-M1_eeg_theta_p2min_norm,ManualStage_has_microwaking_start,ManualStage_has_microwaking_end,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake,ManualStage_AnyDeep
0,1.924170e-09,0.674103,0.536489,0.026340,-0.560567,-0.410648,0.183803,-0.609504,-0.619848,0.291505,...,-0.031615,0.281790,0,0,0.0,0.0,0.0,0.0,1.0,0
1,1.538265e-09,0.742847,0.477407,0.019646,-0.566564,-0.444814,0.214660,-0.605466,-0.598287,0.330733,...,-0.061840,0.096633,0,0,0.0,0.0,0.0,0.0,1.0,0
2,3.222337e-09,0.804278,0.629601,0.015714,-0.569198,-0.469580,0.158740,-0.602967,-0.617150,0.842943,...,-0.076986,0.052729,0,0,0.0,0.0,0.0,0.0,1.0,0
3,1.632114e-09,0.837591,0.583966,0.024473,-0.569084,-0.459612,0.269685,-0.596105,-0.587819,0.307789,...,-0.092663,0.016031,0,0,0.0,0.0,0.0,0.0,1.0,0
4,1.673459e-09,0.880257,0.564774,0.010777,-0.571818,-0.499329,0.156839,-0.593189,-0.597240,0.271290,...,-0.108333,-0.117325,0,0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,5.987828e-11,0.149338,0.181180,0.243420,0.891423,0.906780,1.780555,0.717088,1.047208,0.053974,...,0.079478,-0.051760,0,0,0.0,0.0,0.0,1.0,0.0,0
1090,6.303612e-11,0.172236,0.127809,0.361016,0.961282,1.792029,1.237698,0.730596,1.284927,0.054653,...,0.105994,0.424371,0,0,0.0,0.0,0.0,1.0,0.0,0
1091,1.675323e-10,0.204585,-0.026512,0.154846,0.921318,2.164861,1.208976,0.721860,1.167191,1.215822,...,0.077381,0.643232,1,1,0.0,0.0,0.0,1.0,0.0,0
1092,3.977211e-09,0.252664,0.273996,0.003787,0.833743,1.267842,0.583230,0.686293,0.803938,0.879373,...,0.021929,0.263471,0,0,0.0,0.0,0.0,0.0,1.0,0


,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_at,Fpz-M1_eeg_at_c7min_norm,Fpz-M1_eeg_at_p2min_norm,Fpz-M1_eeg_auc,...,Fpz-M1_eeg_sigma_p2min_norm,Fpz-M1_eeg_skew,Fpz-M1_eeg_skew_c7min_norm,Fpz-M1_eeg_skew_p2min_norm,Fpz-M1_eeg_std,Fpz-M1_eeg_std_c7min_norm,Fpz-M1_eeg_std_p2min_norm,Fpz-M1_eeg_theta,Fpz-M1_eeg_theta_c7min_norm,Fpz-M1_eeg_theta_p2min_norm
0,1.924170e-09,0.674103,0.536489,0.026340,-0.560567,-0.410648,0.183803,-0.609504,-0.619848,0.291505,...,-0.286546,-0.041498,-0.292168,-0.168319,0.000061,0.408875,0.114821,0.143303,-0.031615,0.281790
1,1.538265e-09,0.742847,0.477407,0.019646,-0.566564,-0.444814,0.214660,-0.605466,-0.598287,0.330733,...,-0.314349,-1.054456,-0.295245,-0.300345,0.000072,0.457701,0.133877,0.091520,-0.061840,0.096633
2,3.222337e-09,0.804278,0.629601,0.015714,-0.569198,-0.469580,0.158740,-0.602967,-0.617150,0.842943,...,-0.298760,0.206473,-0.253102,-0.234790,0.000268,0.498781,0.366230,0.098994,-0.076986,0.052729
3,1.632114e-09,0.837591,0.583966,0.024473,-0.569084,-0.459612,0.269685,-0.596105,-0.587819,0.307789,...,-0.271386,-0.612750,-0.229889,-0.255400,0.000069,0.495865,0.310115,0.090746,-0.092663,0.016031
4,1.673459e-09,0.880257,0.564774,0.010777,-0.571818,-0.499329,0.156839,-0.593189,-0.597240,0.271290,...,-0.278903,-0.476831,-0.239482,-0.283770,0.000053,0.505479,0.303755,0.068711,-0.108333,-0.117325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,5.987828e-11,0.149338,0.181180,0.243420,0.891423,0.906780,1.780555,0.717088,1.047208,0.053974,...,0.166258,-0.141322,-0.159096,-0.223251,0.000009,0.731613,0.456701,0.136710,0.079478,-0.051760
1090,6.303612e-11,0.172236,0.127809,0.361016,0.961282,1.792029,1.237698,0.730596,1.284927,0.054653,...,0.497499,-0.034740,-0.153467,-0.196748,0.000009,0.854302,0.230547,0.291684,0.105994,0.424371
1091,1.675323e-10,0.204585,-0.026512,0.154846,0.921318,2.164861,1.208976,0.721860,1.167191,1.215822,...,0.715273,1.139728,-0.145289,-0.121289,0.000454,1.021654,0.320756,0.128080,0.077381,0.643232
1092,3.977211e-09,0.252664,0.273996,0.003787,0.833743,1.267842,0.583230,0.686293,0.803938,0.879373,...,0.406942,-1.350490,-0.146132,-0.182710,0.000230,1.151942,0.510719,0.006493,0.021929,0.263471


,ManualStage_has_microwaking_start,ManualStage_has_microwaking_end,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake,ManualStage_AnyDeep
0,0,0,0.0,0.0,0.0,0.0,1.0,0
1,0,0,0.0,0.0,0.0,0.0,1.0,0
2,0,0,0.0,0.0,0.0,0.0,1.0,0
3,0,0,0.0,0.0,0.0,0.0,1.0,0
4,0,0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...
1089,0,0,0.0,0.0,0.0,1.0,0.0,0
1090,0,0,0.0,0.0,0.0,1.0,0.0,0
1091,1,1,0.0,0.0,0.0,1.0,0.0,0
1092,0,0,0.0,0.0,0.0,0.0,1.0,0


In [9]:
y['ManualStage_AnyDeep'].value_counts()

ManualStage_AnyDeep
0    869
1    221
Name: count, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train

,ManualStage_has_microwaking_start,ManualStage_has_microwaking_end,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake,ManualStage_AnyDeep
573,0,0,0.0,0.0,1.0,0.0,0.0,0
810,0,0,0.0,0.0,1.0,0.0,0.0,0
835,0,0,0.0,0.0,1.0,0.0,0.0,0
918,0,0,0.0,0.0,0.0,0.0,1.0,0
331,0,0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...
330,0,0,0.0,0.0,1.0,0.0,0.0,0
467,0,0,0.0,1.0,0.0,0.0,0.0,1
121,0,0,0.0,0.0,0.0,0.0,1.0,0
1048,0,0,0.0,0.0,1.0,0.0,0.0,0


In [12]:
y_train

,ManualStage_has_microwaking_start,ManualStage_has_microwaking_end,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake,ManualStage_AnyDeep
573,0,0,0.0,0.0,1.0,0.0,0.0,0
810,0,0,0.0,0.0,1.0,0.0,0.0,0
835,0,0,0.0,0.0,1.0,0.0,0.0,0
918,0,0,0.0,0.0,0.0,0.0,1.0,0
331,0,0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...
330,0,0,0.0,0.0,1.0,0.0,0.0,0
467,0,0,0.0,1.0,0.0,0.0,0.0,1
121,0,0,0.0,0.0,0.0,0.0,1.0,0
1048,0,0,0.0,0.0,1.0,0.0,0.0,0


# Training a Catboost model

In [14]:
from catboost import CatBoostClassifier

# Initialize CatBoostClassifier with multi-label support
model = CatBoostClassifier(eval_metric='HammingLoss', loss_function='MultiLogloss', class_names=list(y_train.columns))

# Fit the model
model.fit(X_train, y_train, verbose=100)



Learning rate set to 0.009717
0:	learn: 0.0655103	total: 152ms	remaining: 2m 31s
100:	learn: 0.0368406	total: 2.25s	remaining: 20s
200:	learn: 0.0219323	total: 4.32s	remaining: 17.2s
300:	learn: 0.0150516	total: 6.32s	remaining: 14.7s
400:	learn: 0.0096044	total: 8.23s	remaining: 12.3s
500:	learn: 0.0048739	total: 10.1s	remaining: 10s
600:	learn: 0.0034404	total: 11.9s	remaining: 7.91s
700:	learn: 0.0017202	total: 14.1s	remaining: 6.02s
800:	learn: 0.0014335	total: 16.4s	remaining: 4.08s
900:	learn: 0.0010034	total: 18.9s	remaining: 2.08s
999:	learn: 0.0005734	total: 21.2s	remaining: 0us


In [34]:
test_predict = model.predict(X_test)
test_predict_df = pd.DataFrame(test_predict, columns=y_train.columns)
train_predict = model.predict(X_train)
train_predict_df = pd.DataFrame(train_predict, columns=y_train.columns)

In [38]:
list(y_train.columns)

['ManualStage_has_microwaking_start',
 'ManualStage_has_microwaking_end',
 'ManualStage_Ambiguous Deep',
 'ManualStage_Deep',
 'ManualStage_Non-Deep',
 'ManualStage_Unsure',
 'ManualStage_Wake',
 'ManualStage_AnyDeep']

    # Evaluating the model

In [ ]:
display(X_train)

In [21]:
rejoined = joined.join(y_pred)
# cols_to_drop = [col for col in rejoined.columns if '_true' in col]
# rejoined.drop(columns=cols_to_drop, inplace=True)
rejoined

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [26]:
from sklearn.metrics import accuracy_score

# Convert probabilities to binary predictions
y_pred_list = [(y_pred_prob > 0.5).astype(int) for y_pred_prob in train_predict]

# Calculate accuracy for each target column
accuracy_scores = {col: accuracy_score(y_train[col], y_pred) for col, y_pred in zip(y_train.columns, y_pred_list)}

# Print accuracy scores
for col, score in accuracy_scores.items():
    print(f'Accuracy for {col}: {score}')

ValueError: Found input variables with inconsistent numbers of samples: [872, 8]

In [32]:
display(y_train.shape)
display(train_predict.shape)
display(pd.DataFrame(train_predict, columns=y_train.columns))
train_predict_df = pd.DataFrame(train_predict, columns=y_train.columns)

(872, 8)

(872, 8)

,ManualStage_has_microwaking_start,ManualStage_has_microwaking_end,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake,ManualStage_AnyDeep
0,0,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
867,0,0,0,0,1,0,0,0
868,0,0,0,1,0,0,0,1
869,0,0,0,0,0,0,1,0
870,0,0,0,0,1,0,0,0


In [33]:
from sklearn.metrics import accuracy_score

# Calculate accuracy for each target column
accuracy_scores = {col: accuracy_score(y_train[col], train_predict_df[col]) for col in y_train.columns}

# Print accuracy scores
for col, score in accuracy_scores.items():
    print(f'Accuracy for {col}: {score}')

Accuracy for ManualStage_has_microwaking_start: 1.0
Accuracy for ManualStage_has_microwaking_end: 1.0
Accuracy for ManualStage_Ambiguous Deep: 1.0
Accuracy for ManualStage_Deep: 1.0
Accuracy for ManualStage_Non-Deep: 0.9965596330275229
Accuracy for ManualStage_Unsure: 1.0
Accuracy for ManualStage_Wake: 1.0
Accuracy for ManualStage_AnyDeep: 0.9988532110091743


In [35]:
accuracy_scores = {col: accuracy_score(y_test[col], test_predict_df[col]) for col in y_train.columns}

# Print accuracy scores
for col, score in accuracy_scores.items():
    print(f'Accuracy for {col}: {score}')


Accuracy for ManualStage_has_microwaking_start: 0.9678899082568807
Accuracy for ManualStage_has_microwaking_end: 0.981651376146789
Accuracy for ManualStage_Ambiguous Deep: 0.9862385321100917
Accuracy for ManualStage_Deep: 0.9908256880733946
Accuracy for ManualStage_Non-Deep: 0.9770642201834863
Accuracy for ManualStage_Unsure: 0.981651376146789
Accuracy for ManualStage_Wake: 0.9908256880733946
Accuracy for ManualStage_AnyDeep: 0.9862385321100917


# Saving the model

In [36]:
model.save_model(input_file_without_ext + '.sleep_scoring_catboost_model.cbm')